In [64]:
import pandas as pd

In [65]:
df1 = pd.read_csv('../ecommerce_product_scraping/data/day1_data.csv')
df2 = pd.read_csv(f'../ecommerce_product_scraping/data/day2_data.csv')
df3 = pd.read_csv(f'../ecommerce_product_scraping/data/day3_data.csv')
df4 = pd.read_csv(f'../ecommerce_product_scraping/data/day4_data.csv')

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

print(df.shape)
df.head()

(9484, 30)


,product_id,brand,gender,category,master_category,su_cCategory,article_type,mrp,price,discount,...,has_multiple_sizes,snapshot_date,season,is_fast_fashion,promotion_tags,has_promotion,year,preferred_delivery_tag,delivery_promise,source_sort
0,29615058,TANDUL,Women,Dresses,Apparel,Dress,Dresses,1499,577,922,...,True,2026-01-10,Summer,True,Festive_Price_Crash,True,2024,EXPRESS,Delivery By Jan 12,popularity
1,25497076,W,Women,Dresses,Apparel,Dress,Dresses,4599,2299,2300,...,True,2026-01-10,Fall,True,Festive_Price_Crash|Crazy_Deal,True,2023,EXPRESS,Delivery By Jan 12,popularity
2,20911328,Berrylush Curve,Women,Dresses,Apparel,Dress,Dresses,3499,1084,2415,...,True,2026-01-10,Summer,True,Festive_Price_Crash,True,2025,EXPRESS,Delivery By Jan 12,popularity
3,36145538,Tzelynn,Women,Dresses,Apparel,Dress,Dresses,1499,899,600,...,True,2026-01-10,Summer,True,NaN,False,2025,NaN,NaN,popularity
4,30663177,Berrylush,Women,Dresses,Apparel,Dress,Dresses,2799,727,2072,...,True,2026-01-10,Summer,True,Crazy_Deal|Festive_Price_Crash,True,2025,EXPRESS,Delivery By Jan 12,popularity


In [72]:
df.to_csv('combined_4days.csv')

In [67]:
df.groupby('product_id')['snapshot_date'].nunique().value_counts().sort_index()

snapshot_date
1    1252
2     559
3     938
4    1075
Name: count, dtype: int64

In [68]:
#create new column to check product continuity for every day
product_continuence = df.groupby('product_id')['snapshot_date'].nunique().reset_index(name='days_present')

df = df.merge(product_continuence, on='product_id', how='left')

In [69]:
#create continuity buckets
df['continuence_buckets'] = pd.cut(df['days_present'], bins=[0,1,2,4], labels = ['1_day', '2_day', '3-4_days'])

In [70]:
df.head()

,product_id,brand,gender,category,master_category,su_cCategory,article_type,mrp,price,discount,...,season,is_fast_fashion,promotion_tags,has_promotion,year,preferred_delivery_tag,delivery_promise,source_sort,days_present,continuence_buckets
0,29615058,TANDUL,Women,Dresses,Apparel,Dress,Dresses,1499,577,922,...,Summer,True,Festive_Price_Crash,True,2024,EXPRESS,Delivery By Jan 12,popularity,1,1_day
1,25497076,W,Women,Dresses,Apparel,Dress,Dresses,4599,2299,2300,...,Fall,True,Festive_Price_Crash|Crazy_Deal,True,2023,EXPRESS,Delivery By Jan 12,popularity,3,3-4_days
2,20911328,Berrylush Curve,Women,Dresses,Apparel,Dress,Dresses,3499,1084,2415,...,Summer,True,Festive_Price_Crash,True,2025,EXPRESS,Delivery By Jan 12,popularity,4,3-4_days
3,36145538,Tzelynn,Women,Dresses,Apparel,Dress,Dresses,1499,899,600,...,Summer,True,NaN,False,2025,NaN,NaN,popularity,3,3-4_days
4,30663177,Berrylush,Women,Dresses,Apparel,Dress,Dresses,2799,727,2072,...,Summer,True,Crazy_Deal|Festive_Price_Crash,True,2025,EXPRESS,Delivery By Jan 12,popularity,2,2_day


In [71]:
df.groupby('continuence_buckets')['product_id'].size().value_counts()

/tmp/ipykernel_11563/334171872.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('continuence_buckets')['product_id'].size().value_counts()


product_id
1252    1
1118    1
7114    1
Name: count, dtype: int64